In [47]:
import numpy as np
import torch
from torch.linalg import svd

def construct_hamiltonian(weights):
    """Constructs the Hamiltonian for an unconstrained binary optimization problem."""
    n = len(weights)
    H = torch.zeros((2**n, 2**n))
    
    for i in range(2**n):
        x = torch.tensor([(i >> j) & 1 for j in range(n)])
        H[i, i] = torch.dot(weights, x.float())
    
    return H

def postselection_projector(valid_solutions, n):
    """Constructs a projection operator that projects onto valid constraint-satisfying states."""
    P = torch.zeros((2**n, 2**n))
    for sol in valid_solutions:
        idx = int("".join(map(str, sol)), 2)
        P[idx, idx] = 1.0
    return P

def tensor_network_representation(valid_solutions, n):
    """Constructs a tensor network representation of valid solutions."""
    tensors = []
    for i in range(n):
        Ti = torch.zeros((2, 2))
        for sol in valid_solutions:
            Ti[sol[i], sol[i]] = 1
        tensors.append(Ti)
    return tensors

def imaginary_time_evolution(H, psi_0, tau=1.0, steps=100):
    """Applies imaginary time evolution to find the ground state."""
    psi = psi_0.clone()
    for _ in range(steps):
        psi = torch.mm(torch.matrix_exp(-tau * H), psi)
        psi /= torch.norm(psi)
    return psi

def extract_optimal_solution(psi, n):
    """Finds the optimal binary solution by measuring the expectation of Z operator."""
    best_idx = torch.argmax(torch.abs(psi))
    best_solution = [(best_idx >> i) & 1 for i in range(n)]
    return best_solution

# Example usage
n = 4  # Number of variables
weights = torch.tensor([-1.2, 2.5, -3.0, 0.5])
valid_solutions = [[0, 0, 0, 1], [0, 1, 1, 0], [1, 0, 1, 1]]

H = construct_hamiltonian(weights)
P = postselection_projector(valid_solutions, n)
H_proj = torch.mm(P, torch.mm(H, P))

psi_0 = torch.ones((2**n, 1)) / np.sqrt(2**n)
psi_g = imaginary_time_evolution(H_proj, psi_0)

optimal_solution = extract_optimal_solution(psi_g, n)
print("Optimal Solution:", optimal_solution)


Optimal Solution: [tensor(1), tensor(0), tensor(0), tensor(0)]


In [48]:
import numpy as np
import torch

# Step 1: Define the problem (Binary optimization)
def generate_hamiltonian(weights):
    """Constructs the Hamiltonian matrix H for the optimization problem."""
    n = len(weights)
    dim = 2**n  # Size of the Hamiltonian matrix
    H = torch.zeros((dim, dim), dtype=torch.float32)
    for i in range(n):
        Z = torch.tensor([[1, 0], [0, -1]], dtype=torch.float32)
        I = torch.eye(2, dtype=torch.float32)
        ops = [I] * n
        ops[i] = Z  # Apply Z at the i-th position
        term = ops[0]
        for op in ops[1:]:
            term = torch.kron(term, op)
        H += weights[i] * term
    return H

# Step 2: Define tensor network constraints (simple projector)
def apply_constraints(state, constraints):
    """Projects state onto feasible constraint space."""
    for c in constraints:
        mask = torch.tensor(c, dtype=torch.float32)
        state *= mask
    return state / state.norm()  # Normalize

# Step 3: Imaginary Time Evolution (Gradient Descent-like minimization)
def imaginary_time_evolution(state, H, tau=0.1, steps=100):
    """Performs imaginary time evolution to find the ground state."""
    for _ in range(steps):
        state = torch.matrix_exp(-tau * H) @ state  # Use @ for matrix-vector multiplication
        state /= state.norm()
    return state

# Example: Optimize a simple problem
n = 3  # Number of variables
weights = [-1, -2, -3]  # Example cost function (minimize sum of x)
H = generate_hamiltonian(weights)
initial_state = torch.ones((2**n, 1), dtype=torch.float32) / np.sqrt(2**n)  # Equal superposition
constraints = [torch.randint(0, 2, (2**n, 1))]  # Random binary constraints

# Apply constraints and optimize
state = apply_constraints(initial_state, constraints)
optimal_state = imaginary_time_evolution(state, H)

# Extract best solution
best_solution = (optimal_state < 0).int().squeeze()
print("Optimal solution:", best_solution.tolist())


Optimal solution: [0, 0, 0, 0, 0, 0, 0, 0]


/var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/ipykernel_12767/2439081650.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask = torch.tensor(c, dtype=torch.float32)
